* requirement is that you have the most current version of the QSB and MFR git repositories cloned into the same parent directory

> git clone https://git.ipp-hgw.mpg.de/pih/bolometer_mfr
> -------------------------------------------------------

> git clone https://git.ipp-hgw.mpg.de/pih/QSB_Bolometry
> -------------------------------------------------------

* given that this worked, you will be able to compile the 2D & 3D grid, geometry data and info

## you might need the IDLlib from NASA

* go to https://idlastro.gsfc.nasa.gov/idlfaq.html#A3

* in Window>User Preferences>IDL>Paths you add the location where you extracted the library

* change the appropriate base directory where you want to work in

In [ ]:
import os, sys, importlib

# need to change directories
libdir = '//share.ipp-hgw.mpg.de//documents//pih//Documents//git//QSB_Bolometry//'
root = '//share.ipp-hgw.mpg.de//documents//pih//Documents//git//QSB_Bolometry//libprad//'

os.chdir(root)
print('\\\ working dir:', os.getcwd())

QSB Libraries
-------------

In [ ]:
sys.path.append(libdir + 'files/customs/')
sys.path.append(libdir + 'libprad/')
sys.path.append(libdir + 'libprad/libwebapi/')
sys.path.append(libdir + 'libprad/liboutput/')
sys.path.append(libdir + 'libprad/libtraining/')
sys.path.append(libdir + 'libprad/libradcalc/')
sys.path.append(libdir + 'libprad/libscaling/')
print('\\\ loaded QSB lib')

Inversion libraries
-------------------

In [ ]:
sys.path.append(libdir + 'libinversion/')
sys.path.append(libdir + 'libinversion/libcalc')
sys.path.append(libdir + 'libinversion/libaccessoires/')
sys.path.append(libdir + 'libinversion/liboutput/')
print('\\\ loaded inversion lib')

* available magnetic configurations in database definition

In [ ]:
import database_inv
foo = database_inv.import_database()

for i, conf in enumerate(foo['values']['magnetic_configurations'].keys()):
    print(conf)  #  + ':','\n\t', foo['values']['magnetic_configurations'][conf]['description'])

In [ ]:
program = '20181010.032'
nPhi = 10  # toroidal slices
nFS = 10  # radial slices
nL = 10  # poloidal slices
vpF = 1.3  # blowup from LCFS
N = [1, 2]  # detector seperation, short X long side, 8
nN = N[0] * N[1] if isinstance(N, list) else N
interpolation = 'square'  # detector split mode (triang)
magconf = 'EIM_000'  # magnetic configuration

strgrid = interpolation[0] + 'N' + str(nN) + '_' + \
          str(nPhi) + 'x' + str(nFS) + 'x' + str(nL) + '_' + str(vpF) 
confstr = magconf[:4] + 'beta' + magconf[-3:] + '_' + interpolation[0] + \
          'N' + str(nN) + '_' + str(nPhi) + 'x' + str(nFS) + \
          'x' + str(nL) + '_' + str(vpF)
print('\\\ strgrid ' + strgrid + ' confstr ' + confstr)

def crtdir(loc):
    try:
        os.mkdir(loc)
    except Exception:
        print('\\\ ' + loc + ' exists')
    return

os.chdir(root)
crtdir(root + '../results')
crtdir(root + '../results/CURRENT/')
crtdir(root + '../results/INVERSION/')

crtdir(root + '../results/INVERSION/PROFILE')
crtdir(root + '../results/INVERSION/PROFILE/' + confstr)

crtdir(root + '../results/INVERSION/CAMGEO')
crtdir(root + '../results/INVERSION/FS')
crtdir(root + '../results/INVERSION/MESH')
crtdir(root + '../results/INVERSION/MESH/TRIAGS')

crtdir(root + '../results/INVERSION/MFR')
crtdir(root + '../results/INVERSION/MFR/' + strgrid)

os.chdir(root)
print('\\\ working dir:', os.getcwd())

* coming up is the routine that calculates the geometry data in 2D and 3D including volume, emissivity, radius (effective in cross section)

* input parameters are pretty much self explanatory, directories specific to the combination of input might have to be created beforehand

> ../results/INVERSION/PROFILE/ + 'magconf' + _ + 's/t' + N + 'N\[0\]*N\[1] / N' + _ + 'nPhi' + x + 'nFS' + x + 'nL' + _ + 'vpF'

In [ ]:
import invert_main as invert
importlib.reload(invert)
invert.main(
    nPhi=nPhi,
    nFS=nFS,
    nL=nL,
    vpF=vpF,
    N=[1, 2],
    interp_method=interpolation,    
    VMID=magconf,
    tilt=False,  # tilt camera
    tilt_deg=.0,  # camera aperture tilt if enabled
    random_error=False,  # wiggle at apertureas
    error_scale=0.0001,  # aperture positioning error (rand, mm)
    add_camera=False,  # add artificial camera
    new_type=None,  # artificial camera (HBCm)
    cartesian=False,  # cartesian grid
    artificial_HBCm=False,  # perfectly symmetric HBCm camera
    fix_LoS=False,  # LoS fix up/down asymmetry
    centered=False,  # centered HBCm aperture to z=0.0
    symmetric=False,  # symmetric around aperture normal?
    debug=True,  # verbose
    plot=False)  # plot after

* now one needs to reformat and calculate the important values for the IDL interface and MFR method

In [ ]:
import mfr2D_matrix_gridtransform as transf

crtdir(root + '../../bolometer_mfr/chpow/' + program)
print('>> Make files for MFR routine from 3D/2D results')
transf.prepare_geometry_power(
    program=program,
    label='_' + confstr,
    strgrid=strgrid,
    debug=False)

* next comes the set of commands you need to do the mfr routine

### after opening the IDL console and you do:

* moving to the source code directory, change *pih* to something accordingly of your usr

> IDL> CD, '\\share.ipp-hgw.mpg.de\documents\pih\Documents\git\bolometer_mfr\source'

* compiling the routine

> IDL> .compile mfrmain19

* main mfr routine, true or false values in input are given via 1/0


> IDL>    
    mfr,
    shot='20181010.032',  ; program    
    magconf='EIM_beta000',  ; magnetic configuration    
    phantom=0,  ; phantom type string if phantom test, else true/false    
    strgrid='sN2_10x10x10_1.3',  ; grid string from before    
    artificial=0,  ; artificial camera true/false    
    new_type='',  ; new camera type if true, 'HBCm'    
    reduced=1,  ; reduced set of lines of sight for XP, 0 for phantoms    
    RGS=0,  ; RGS method, true/false   
    abel=0,  ; linear inversion if abel, true/false    
    nigs=1,  ; number of gaussian error pertubation tests after inversion    
    no_ani=3,  ; modes of K factor profiles, see below   
    kcore=2.,  ; inside (small radii) K factor    
    kedge=.35,  ; outside (large radii) K factor    
    nTarget=7,  ; profile point to change in-/outside    
    nWidth=1,  ; width of transition in K profile for no_ani=4    
    tran_inv=\[1., 1.05\],  ; point in time/time trace to invert, steps at 0.05s, minimum interval    
    fac_err=1.,  ; error weighting factor    
    debug=1  ; verbose printing and imaging

### EXAMPLE

> IDL> mfr,shot='20181010.032',magconf='EIM_beta000',strgrid='sN2_10x10x10_1.3',debug=0,no_ani=3,kcore=2.,kedge=.35,nTarget=7,tran_inv=\[2.8, 2.85\]

### K Factor Modes

* no_ani: 0
> ncore = 5    
  kani\[0:ncore - 1\] = kcore    
  kani\[ncore:nrmax - 1\] = (kcore - (INDGEN(next) + 1.) * kedge / next)

* no_ani: 1
> kani = 1.

* no_ani: 2
> kani\[0:ncore - 1\] = kcore   
  kani\[ncore:nrmax - 1\] = (kcore - (INDGEN(next) + 1.) * kedge / next)

* no_ani: 3
> kani\[0:nTarget\] = kcore    
  kani\[nTarget:nrmax - 1\] = kedge

* no_ani: 4
> kani = smooth arctan around nTarget with nWidth transition

## Visualization

* after the MFR routine finished plot and diagnose all the stuff

In [ ]:
import mfr2D_accessoires as mfr2D
os.chdir(root)
print('\\\ working dir:', os.getcwd())

results = mfr2D.get_mfr_results(
    program=program,
    no_ani=3,  # no_ani mode
    kani=[2., .35],  # [kcore, kedge]
    nVals=7,  # nTarget, or [nTarget, nWidth]
    nigs=1,
    times=1.,
    grid_nt=10,
    grid_nr=10,
    strgrid=strgrid,
    base='_mfr1D',
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    new_type=None,
    add_camera=False,
    reduced=True,
    phantom=False,
    plot=True,
    saving=True,
    debug=False)

## Phantoms
* you wanna do phantom images to reconstruct you have to contruct them in the first place

### possibilities in mfr_phantoms (comment/uncomment)
* STRAHL_to_phantom(strahl_id='00091',...)

* spot_pos_mesh(position=...)

* radial_profile(radius, sigma1, sigma2, ....)

* fluxsurface_reff_ring(radius, sigma, ...)

* anisotropic_fluxsurface_ring(radius, mode_number, ...)

* blind_test_ones(all at max level)

In [ ]:
import mfr_phantoms as mfrp
os.chdir(root)
print('\\\ working dir:', os.getcwd())

mfrp.ch_pow_phantoms(
    strgrid=strgrid,
    sigma=0.2,  # smoothness in phantom, in (W/m^3)/m
    x0=[5., 0.],  # hot spot location if chosen, in m
    amplitude=1.e6,  # maximum, in W/m^3
    add_noise=False,  #  random error
    error=0.025,  # in % of max amplitude, W/m^3
    in_label='_' + confstr,
    save_base='../results/INVERSION/MFR/',
    new_type=None,  # new artificial camera type 
    add_camera=False,  # artificial camera
    systematic_error=False,  # camera geometry error implemented
    debug=False)  # debugging

### MFR Phantom Example
* the new mfr call should look like
> IDL>    
mfr,    
shot='phan_',    
phantom='sym_R1.1_m5_mx1.0e+06',    
magconf='EIM_beta000',    
strgrid='sN2_10x10x10_1.3',    
debug=1,    
no_ani=3,    
kcore=.1,    
kedge=2.,    
nTarget=7,    
tran_inv=\[0.11, 0.16\]


## phantom comparison visualization


In [ ]:
import phantom_metrics as phm
os.chdir(root)
print('\\\ working dir:', os.getcwd())

results = phm.compare_tomo_phantom(
    nFS=nFS,
    nL=nL,
    N=nN,
    no_ani=3,
    kani=[.1, 2.],
    nVals=7,
    nigs=1,
    times=0.11,
    label='sym_R1.1_m5_mx1.0e+06',
    strgrid=strgrid,
    base='_mfr1D',
    vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False,
    new_type=None,
    reduced=False,
    plot=True,
    saving=True,
    debug=False)